In [19]:
import tensorflow as tf
import os
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


/home/rohit/imivenv/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['seed']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
env = 'MiniGrid-DoorKey-6x6-v0'
disctype = 'DoorKeyGAILpositive'
seed=1
disc_dir = '/serverdata/rohit/reward_bias/imitation/output/train_adversarial/{}/{}/{}/checkpoints/final/discrim'.format(env, disctype, seed)

In [13]:
list(os.walk(disc_dir))

[('/serverdata/rohit/reward_bias/imitation/output/train_adversarial/MiniGrid-DoorKey-6x6-v0/DoorKeyGAILpositive/1/checkpoints/final/discrim',
  [],
  ['weights-1.index',
   'loader',
   'weights-1.data-00000-of-00002',
   'weights-1.data-00001-of-00002',
   'checkpoint',
   'obj'])]

In [14]:
from imitation.rewards import discrim_net

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [68]:
with tf.variable_scope('test'):
    model = discrim_net.DiscrimNetGAIL.load(disc_dir)

Using CNN for discriminator
Tensor("test/discrim_network/LeakyRelu_1:0", shape=(?, 2, 2, 64), dtype=float32)
INFO:tensorflow:using GAIL


In [69]:
model.reward_test?

In [26]:
import gym
import gym_minigrid
from gym_minigrid import wrappers as mgwr
env = gym.make('MiniGrid-Empty-Random-6x6-v0')
env = mgwr.FullyObsWrapper(env)
env = mgwr.ImgObsWrapper(env)
env = mgwr.FullyObsOneHotWrapper(env, drop_color=1, keep_classes=['goal', 'agent', 'wall', 'empty', 'door', 'key'], flatten=False)

In [36]:
import pickle as pkl
with open('/serverdata/rohit/reward_bias/imitation/output/expert_demos/MiniGrid-DoorKey-6x6-v0/DoorKeyPPO/rollouts/final.pkl', 'rb') as fi:
    rollouts = pkl.load(fi)

In [80]:
obs = rollouts[0].obs[:-1]
next_obs = rollouts[0].obs[1:]
act = rollouts[0].acts

In [90]:
with tf.variable_scope('test'):
    model._sess = tf.Session()
    model._sess.run(tf.global_variables_initializer())
    print(model.reward_test(obs, act, next_obs, None))

InvalidArgumentError: You must feed a value for placeholder tensor 'test/obs' with dtype uint8 and shape [?,6,6,10]
	 [[node test/obs (defined at /home/rohit/imivenv/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'test/obs':
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/rohit/imivenv/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/rohit/imivenv/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/rohit/imivenv/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 583, in start
    self.io_loop.start()
  File "/home/rohit/imivenv/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  

(10,)